In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd

from tensorflow.keras.layers import Dense, LSTM, Input, TimeDistributed, Bidirectional, Activation
from tensorflow.keras.regularizers import l2
from tensorflow.keras.models import Model

In [2]:
# bd sn ht mt lt oh ch cr ri
checking_arr = [
    [1, 0, 0, 0, 0, 0, 0, 0, 0], #0
    [0, 1, 0, 0, 0, 0, 0, 0, 0], #1
    # [0, 0, 1, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 1, 0, 0, 0, 0, 0], #2
    [0, 0, 0, 0, 1, 0, 0, 0, 0], #3
    [0, 0, 0, 0, 0, 1, 0, 0, 0], #4
    [0, 0, 0, 0, 0, 0, 1, 0, 0], #5
    # [0, 0, 0, 0, 0, 0, 0, 1, 0], 
    [0, 0, 0, 0, 0, 0, 0, 0, 1], #6
    [1, 0, 0, 0, 0, 0, 1, 0, 0], #7
    [1, 0, 0, 0, 0, 1, 0, 0, 0], #8
    [1, 0, 0, 0, 0, 0, 0, 0, 1], #9
    [1, 0, 0, 0, 1, 0, 0, 0, 0], #10
    [1, 0, 0, 0, 0, 0, 0, 1, 0], #11
    [1, 1, 0, 0, 0, 0, 0, 0, 0], #12
    [0, 1, 0, 0, 0, 0, 1, 0, 0], #13
    [0, 1, 0, 0, 0, 1, 0, 0, 0], #14
    [0, 1, 0, 0, 0, 0, 0, 0, 1], #15
    [0, 1, 0, 0, 1, 0, 0, 0, 0], #16
    [0, 1, 0, 0, 0, 0, 0, 1, 0], #17
    [0, 0, 0, 1, 1, 0, 0, 0, 0], #18
    [1, 1, 0, 0, 0, 0, 1, 0, 0], #19
    [1, 1, 0, 0, 0, 1, 0, 0, 0], #20
    [1, 1, 0, 0, 0, 0, 0, 0, 1], #21
    [0, 0, 0, 0, 0, 0, 0, 0, 0] #22
]


seven_kind_arr = [
   # bd sn mt lt oh ch ri 
    [1, 0, 0, 0, 0, 0, 0], #0
    [0, 1, 0, 0, 0, 0, 0], #1
    [0, 0, 1, 0, 0, 0, 0], #2
    [0, 0, 0, 1, 0, 0, 0], #3
    [0, 0, 0, 0, 1, 0, 0], #4
    [0, 0, 0, 0, 0, 1, 0], #5
    [0, 0, 0, 0, 0, 0, 1], #6
    [1, 0, 0, 0, 0, 1, 0], #7
    [1, 0, 0, 0, 1, 0, 0], #8
    [1, 0, 0, 0, 0, 0, 1], #9
    [1, 0, 0, 1, 0, 0, 0], #10
    [1, 1, 0, 0, 0, 0, 0], #11
    [0, 1, 0, 0, 0, 1, 0], #12
    [0, 1, 0, 0, 1, 0, 0], #13
    [0, 1, 0, 0, 0, 0, 1], #14
    [0, 1, 0, 1, 0, 0, 0], #15
    [0, 1, 0, 0, 0, 0, 1], #16
    [0, 0, 1, 1, 0, 0, 0], #17
    [1, 1, 0, 0, 0, 1, 0], #18
    [1, 1, 0, 0, 1, 0, 0], #19
    [1, 1, 0, 0, 0, 0, 1], #20
    [0, 0, 0, 0, 0, 0, 0]
]




In [3]:
def return_pattern(i):
    return seven_kind_arr[i.tolist().index(1)]

In [47]:
train_x = []
train_y = []

for i in range(1,6000):
    tmp = pd.read_csv('./single_label_dataset/' + str(i) + '.csv.gzip', compression='gzip').to_numpy()
    imsi = []
    for j in range(63):
        imsi.append(return_pattern(tmp[j]))
    train_x.append(imsi)
    train_y.append(return_pattern(tmp[-1]))

train_x = np.array(train_x)
train_y = np.array(train_y)



In [5]:
y1 = np.swapaxes(train_y,0,1)[0]
y2 = np.swapaxes(train_y,0,1)[1]
y3 = np.swapaxes(train_y,0,1)[2]
y4 = np.swapaxes(train_y,0,1)[3]
y5 = np.swapaxes(train_y,0,1)[4]
y6 = np.swapaxes(train_y,0,1)[5]
y7 = np.swapaxes(train_y,0,1)[6]
# y8 = np.swapaxes(train_y,0,1)[7]
# y9 = np.swapaxes(train_y,0,1)[8]

In [56]:
def create_model(drp):
    inputt = Input(shape=(63,7))
    
    lstm1 = Bidirectional(LSTM(512, return_sequences=True, dropout=drp , recurrent_dropout=drp))(inputt)
    lstm2 = Bidirectional(LSTM(128, return_sequences=True, dropout=drp , recurrent_dropout=drp))(lstm1)
    lstm3 = Bidirectional(LSTM(32, dropout=drp , recurrent_dropout=drp))(lstm2)

    out1 = Dense(1, activation='sigmoid', name='bd')(lstm3)
    out2 = Dense(1, activation='sigmoid', name='sn')(lstm3)
    out3 = Dense(1, activation='sigmoid', name='mt')(lstm3)
    out4 = Dense(1, activation='sigmoid', name='lt')(lstm3)
    out5 = Dense(1, activation='sigmoid', name='oh')(lstm3)
    out6 = Dense(1, activation='sigmoid', name='ch')(lstm3)
    out7 = Dense(1, activation='sigmoid', name='ri')(lstm3)
    # out8 = Dense(1, activation='sigmoid', name='cr')(lstm3)
    # out9 = Dense(1, activation='sigmoid', name='ri')(lstm3)

    # return Model(inputs=inputt, outputs=[out1,out2,out3,out4,out5,out6,out7,out8,out9])
    return Model(inputs=inputt, outputs=[out1,out2,out3,out4,out5,out6,out7])

In [9]:
# model = create_model(0.1)

callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=6)

In [63]:
model.summary()

Model: "model_9"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_10 (InputLayer)           [(None, 63, 7)]      0                                            
__________________________________________________________________________________________________
bidirectional_27 (Bidirectional (None, 63, 1024)     2129920     input_10[0][0]                   
__________________________________________________________________________________________________
bidirectional_28 (Bidirectional (None, 63, 256)      1180672     bidirectional_27[0][0]           
__________________________________________________________________________________________________
bidirectional_29 (Bidirectional (None, 64)           73984       bidirectional_28[0][0]           
____________________________________________________________________________________________

In [64]:
model.compile(loss = tf.keras.losses.BinaryCrossentropy(), optimizer=tf.keras.optimizers.RMSprop(), metrics=['accuracy'])

In [65]:
with tf.device("/gpu:0"):
    model.fit(x=train_x,y=[y1,y2,y3,y4,y5,y6,y7], epochs=4000, callbacks=callback, validation_split=0.1)

Epoch 1/4000
169/169 [==============================] - 393s 2s/step - loss: 2.2973 - bd_loss: 0.5422 - sn_loss: 0.6355 - mt_loss: 0.0742 - lt_loss: 0.0551 - oh_loss: 0.1765 - ch_loss: 0.6451 - ri_loss: 0.1688 - bd_accuracy: 0.7694 - sn_accuracy: 0.6744 - mt_accuracy: 0.9868 - lt_accuracy: 0.9874 - oh_accuracy: 0.9591 - ch_accuracy: 0.6197 - ri_accuracy: 0.9587 - val_loss: 2.2560 - val_bd_loss: 0.5369 - val_sn_loss: 0.6461 - val_mt_loss: 0.0833 - val_lt_loss: 0.0720 - val_oh_loss: 0.1638 - val_ch_loss: 0.5864 - val_ri_loss: 0.1676 - val_bd_accuracy: 0.7717 - val_sn_accuracy: 0.6833 - val_mt_accuracy: 0.9833 - val_lt_accuracy: 0.9867 - val_oh_accuracy: 0.9617 - val_ch_accuracy: 0.7050 - val_ri_accuracy: 0.9550
Epoch 2/4000
169/169 [==============================] - 387s 2s/step - loss: 2.1008 - bd_loss: 0.5389 - sn_loss: 0.5916 - mt_loss: 0.0664 - lt_loss: 0.0402 - oh_loss: 0.1674 - ch_loss: 0.5634 - ri_loss: 0.1330 - bd_accuracy: 0.7731 - sn_accuracy: 0.7083 - mt_accuracy: 0.9876 - lt_

In [66]:
model.save('./chooseok/multi_output_1_drp0.1.h5')

In [67]:
model = create_model(0.2)
model.compile(loss = tf.keras.losses.BinaryCrossentropy(), optimizer=tf.keras.optimizers.RMSprop(), metrics=['accuracy'])

In [68]:
with tf.device("/gpu:0"):
    model.fit(x=train_x,y=[y1,y2,y3,y4,y5,y6,y7], epochs=4000, callbacks=callback, validation_split=0.1)

Epoch 1/4000
169/169 [==============================] - 383s 2s/step - loss: 2.3095 - bd_loss: 0.5445 - sn_loss: 0.6346 - mt_loss: 0.0734 - lt_loss: 0.0546 - oh_loss: 0.1771 - ch_loss: 0.6545 - ri_loss: 0.1707 - bd_accuracy: 0.7703 - sn_accuracy: 0.6670 - mt_accuracy: 0.9837 - lt_accuracy: 0.9887 - oh_accuracy: 0.9568 - ch_accuracy: 0.6181 - ri_accuracy: 0.9559 - val_loss: 2.2261 - val_bd_loss: 0.5424 - val_sn_loss: 0.6377 - val_mt_loss: 0.0831 - val_lt_loss: 0.0675 - val_oh_loss: 0.1626 - val_ch_loss: 0.5770 - val_ri_loss: 0.1558 - val_bd_accuracy: 0.7717 - val_sn_accuracy: 0.6833 - val_mt_accuracy: 0.9833 - val_lt_accuracy: 0.9867 - val_oh_accuracy: 0.9617 - val_ch_accuracy: 0.6583 - val_ri_accuracy: 0.9550
Epoch 2/4000
169/169 [==============================] - 376s 2s/step - loss: 2.1699 - bd_loss: 0.5412 - sn_loss: 0.6178 - mt_loss: 0.0650 - lt_loss: 0.0450 - oh_loss: 0.1714 - ch_loss: 0.5823 - ri_loss: 0.1472 - bd_accuracy: 0.7733 - sn_accuracy: 0.6833 - mt_accuracy: 0.9876 - lt_

In [69]:
model.save('./chooseok/multi_output_1_drp0.2.h5')

In [70]:
model = create_model(0.4)
model.compile(loss = tf.keras.losses.BinaryCrossentropy(), optimizer=tf.keras.optimizers.RMSprop(), metrics=['accuracy'])

In [71]:
with tf.device("/gpu:0"):
    model.fit(x=train_x,y=[y1,y2,y3,y4,y5,y6,y7], epochs=4000, callbacks=callback, validation_split=0.1)

Epoch 1/4000
169/169 [==============================] - 388s 2s/step - loss: 2.3017 - bd_loss: 0.5432 - sn_loss: 0.6359 - mt_loss: 0.0748 - lt_loss: 0.0547 - oh_loss: 0.1761 - ch_loss: 0.6570 - ri_loss: 0.1600 - bd_accuracy: 0.7726 - sn_accuracy: 0.6768 - mt_accuracy: 0.9856 - lt_accuracy: 0.9911 - oh_accuracy: 0.9550 - ch_accuracy: 0.6116 - ri_accuracy: 0.9572 - val_loss: 2.2836 - val_bd_loss: 0.5410 - val_sn_loss: 0.6241 - val_mt_loss: 0.0900 - val_lt_loss: 0.0796 - val_oh_loss: 0.1626 - val_ch_loss: 0.6167 - val_ri_loss: 0.1697 - val_bd_accuracy: 0.7717 - val_sn_accuracy: 0.6833 - val_mt_accuracy: 0.9833 - val_lt_accuracy: 0.9867 - val_oh_accuracy: 0.9617 - val_ch_accuracy: 0.6583 - val_ri_accuracy: 0.9550
Epoch 2/4000
169/169 [==============================] - 383s 2s/step - loss: 2.2036 - bd_loss: 0.5413 - sn_loss: 0.6242 - mt_loss: 0.0684 - lt_loss: 0.0477 - oh_loss: 0.1724 - ch_loss: 0.6090 - ri_loss: 0.1406 - bd_accuracy: 0.7733 - sn_accuracy: 0.6768 - mt_accuracy: 0.9876 - lt_

In [72]:
model.save('./chooseok/multi_output_1_drp0.4.h5')

In [73]:
def create_other_model(drp):
    inputt = Input(shape=(63,7))
    
    lstm1 = Bidirectional(LSTM(512, return_sequences=True, dropout=drp , recurrent_dropout=drp))(inputt)
    lstm2 = Bidirectional(LSTM(128, return_sequences=True, dropout=drp , recurrent_dropout=drp))(lstm1)

    lstm_1 = Bidirectional(LSTM(32, dropout=drp , recurrent_dropout=drp))(lstm2)
    out1 = Dense(1, activation='sigmoid', name='bd')(lstm_1)
    
    lstm_2 = Bidirectional(LSTM(32, dropout=drp , recurrent_dropout=drp))(lstm2)
    out2 = Dense(1, activation='sigmoid', name='sn')(lstm_2)

    lstm_3 = Bidirectional(LSTM(32, dropout=drp , recurrent_dropout=drp))(lstm2)
    out3 = Dense(1, activation='sigmoid', name='mt')(lstm_3)

    lstm_4 = Bidirectional(LSTM(32, dropout=drp , recurrent_dropout=drp))(lstm2)
    out4 = Dense(1, activation='sigmoid', name='lt')(lstm_4)

    lstm_5 = Bidirectional(LSTM(32, dropout=drp , recurrent_dropout=drp))(lstm2)
    out5 = Dense(1, activation='sigmoid', name='oh')(lstm_5)

    lstm_6 = Bidirectional(LSTM(32, dropout=drp , recurrent_dropout=drp))(lstm2)
    out6 = Dense(1, activation='sigmoid', name='ch')(lstm_6)

    lstm_7 = Bidirectional(LSTM(32, dropout=drp , recurrent_dropout=drp))(lstm2)
    out7 = Dense(1, activation='sigmoid', name='ri')(lstm_7)
    # out8 = Dense(1, activation='sigmoid', name='cr')(lstm3)
    # out9 = Dense(1, activation='sigmoid', name='ri')(lstm3)

    # return Model(inputs=inputt, outputs=[out1,out2,out3,out4,out5,out6,out7,out8,out9])
    return Model(inputs=inputt, outputs=[out1,out2,out3,out4,out5,out6,out7])

In [74]:
model = create_model(0.2)
model.compile(loss = tf.keras.losses.BinaryCrossentropy(), optimizer=tf.keras.optimizers.RMSprop(), metrics=['accuracy'])

In [75]:
model = create_other_model(0.2)
model.compile(loss = tf.keras.losses.BinaryCrossentropy(), optimizer=tf.keras.optimizers.RMSprop(), metrics=['accuracy'])

In [76]:
with tf.device("/gpu:0"):
    model.fit(x=train_x,y=[y1,y2,y3,y4,y5,y6,y7], epochs=4000, callbacks=callback, validation_split=0.1)

Epoch 1/4000
169/169 [==============================] - 1267s 7s/step - loss: 2.2921 - bd_loss: 0.5501 - sn_loss: 0.6307 - mt_loss: 0.0752 - lt_loss: 0.0548 - oh_loss: 0.1778 - ch_loss: 0.6416 - ri_loss: 0.1619 - bd_accuracy: 0.7709 - sn_accuracy: 0.6759 - mt_accuracy: 0.9865 - lt_accuracy: 0.9856 - oh_accuracy: 0.9585 - ch_accuracy: 0.6234 - ri_accuracy: 0.9537 - val_loss: 2.1104 - val_bd_loss: 0.5355 - val_sn_loss: 0.5650 - val_mt_loss: 0.0825 - val_lt_loss: 0.0656 - val_oh_loss: 0.1522 - val_ch_loss: 0.5645 - val_ri_loss: 0.1450 - val_bd_accuracy: 0.7717 - val_sn_accuracy: 0.7633 - val_mt_accuracy: 0.9833 - val_lt_accuracy: 0.9867 - val_oh_accuracy: 0.9617 - val_ch_accuracy: 0.6583 - val_ri_accuracy: 0.9550
Epoch 2/4000
169/169 [==============================] - 1221s 7s/step - loss: 2.0130 - bd_loss: 0.5371 - sn_loss: 0.5599 - mt_loss: 0.0652 - lt_loss: 0.0386 - oh_loss: 0.1594 - ch_loss: 0.5331 - ri_loss: 0.1198 - bd_accuracy: 0.7729 - sn_accuracy: 0.7301 - mt_accuracy: 0.9870 - l

In [77]:
model.save('./chooseok/multi_output_2_drp0.2.h5')

In [78]:
model = create_other_model(0.4)
model.compile(loss = tf.keras.losses.BinaryCrossentropy(), optimizer=tf.keras.optimizers.RMSprop(), metrics=['accuracy'])

In [79]:
with tf.device("/gpu:0"):
    model.fit(x=train_x,y=[y1,y2,y3,y4,y5,y6,y7], epochs=4000, callbacks=callback, validation_split=0.1)

Epoch 1/4000
169/169 [==============================] - 1250s 7s/step - loss: 2.3160 - bd_loss: 0.5476 - sn_loss: 0.6380 - mt_loss: 0.0744 - lt_loss: 0.0552 - oh_loss: 0.1780 - ch_loss: 0.6530 - ri_loss: 0.1697 - bd_accuracy: 0.7696 - sn_accuracy: 0.6705 - mt_accuracy: 0.9844 - lt_accuracy: 0.9891 - oh_accuracy: 0.9563 - ch_accuracy: 0.6027 - ri_accuracy: 0.9563 - val_loss: 2.2292 - val_bd_loss: 0.5397 - val_sn_loss: 0.6292 - val_mt_loss: 0.0824 - val_lt_loss: 0.0715 - val_oh_loss: 0.1633 - val_ch_loss: 0.5720 - val_ri_loss: 0.1710 - val_bd_accuracy: 0.7717 - val_sn_accuracy: 0.6833 - val_mt_accuracy: 0.9833 - val_lt_accuracy: 0.9867 - val_oh_accuracy: 0.9617 - val_ch_accuracy: 0.6433 - val_ri_accuracy: 0.9550
Epoch 2/4000
169/169 [==============================] - 1199s 7s/step - loss: 2.1343 - bd_loss: 0.5385 - sn_loss: 0.5995 - mt_loss: 0.0656 - lt_loss: 0.0433 - oh_loss: 0.1722 - ch_loss: 0.5836 - ri_loss: 0.1317 - bd_accuracy: 0.7733 - sn_accuracy: 0.6966 - mt_accuracy: 0.9874 - l

KeyboardInterrupt: 

In [ ]:
model.save('./chooseok/multi_output_2_drp0.4.h5')

In [6]:
def create_l2_model(drp):
    inputt = Input(shape=(63,7))
    
    lstm1 = Bidirectional(LSTM(512, return_sequences=True, dropout=drp , recurrent_dropout=drp, kernel_regularizer=l2(0.01), recurrent_regularizer=l2(0.01)))(inputt)
    lstm2 = Bidirectional(LSTM(128, return_sequences=True, dropout=drp , recurrent_dropout=drp, kernel_regularizer=l2(0.01), recurrent_regularizer=l2(0.01)))(lstm1)

    lstm_1 = Bidirectional(LSTM(32, dropout=drp , recurrent_dropout=drp))(lstm2)
    out1 = Dense(1, activation='sigmoid', name='bd')(lstm_1)
    
    lstm_2 = Bidirectional(LSTM(32, dropout=drp , recurrent_dropout=drp))(lstm2)
    out2 = Dense(1, activation='sigmoid', name='sn')(lstm_2)

    lstm_3 = Bidirectional(LSTM(32, dropout=drp , recurrent_dropout=drp))(lstm2)
    out3 = Dense(1, activation='sigmoid', name='mt')(lstm_3)

    lstm_4 = Bidirectional(LSTM(32, dropout=drp , recurrent_dropout=drp))(lstm2)
    out4 = Dense(1, activation='sigmoid', name='lt')(lstm_4)

    lstm_5 = Bidirectional(LSTM(32, dropout=drp , recurrent_dropout=drp))(lstm2)
    out5 = Dense(1, activation='sigmoid', name='oh')(lstm_5)

    lstm_6 = Bidirectional(LSTM(32, dropout=drp , recurrent_dropout=drp))(lstm2)
    out6 = Dense(1, activation='sigmoid', name='ch')(lstm_6)

    lstm_7 = Bidirectional(LSTM(32, dropout=drp , recurrent_dropout=drp))(lstm2)
    out7 = Dense(1, activation='sigmoid', name='ri')(lstm_7)
    # out8 = Dense(1, activation='sigmoid', name='cr')(lstm3)
    # out9 = Dense(1, activation='sigmoid', name='ri')(lstm3)

    # return Model(inputs=inputt, outputs=[out1,out2,out3,out4,out5,out6,out7,out8,out9])
    return Model(inputs=inputt, outputs=[out1,out2,out3,out4,out5,out6,out7])

In [20]:
model = create_l2_model(0.2)
model.compile(loss = tf.keras.losses.BinaryCrossentropy(), optimizer=tf.keras.optimizers.Adam(), metrics=['accuracy'])

In [21]:
with tf.device("/gpu:0"):
    model.fit(x=train_x,y=[y1,y2,y3,y4,y5,y6,y7], epochs=4000, callbacks=callback, validation_split=0.1)

Epoch 1/4000
169/169 [==============================] - 1316s 8s/step - loss: 5.3360 - bd_loss: 0.5424 - sn_loss: 0.5975 - mt_loss: 0.0777 - lt_loss: 0.0550 - oh_loss: 0.1784 - ch_loss: 0.5740 - ri_loss: 0.1462 - bd_accuracy: 0.7718 - sn_accuracy: 0.6999 - mt_accuracy: 0.9828 - lt_accuracy: 0.9872 - oh_accuracy: 0.9593 - ch_accuracy: 0.6786 - ri_accuracy: 0.9567 - val_loss: 2.1774 - val_bd_loss: 0.5361 - val_sn_loss: 0.5535 - val_mt_loss: 0.0855 - val_lt_loss: 0.0753 - val_oh_loss: 0.1620 - val_ch_loss: 0.5193 - val_ri_loss: 0.1488 - val_bd_accuracy: 0.7717 - val_sn_accuracy: 0.7450 - val_mt_accuracy: 0.9833 - val_lt_accuracy: 0.9867 - val_oh_accuracy: 0.9617 - val_ch_accuracy: 0.7217 - val_ri_accuracy: 0.9533
Epoch 2/4000
169/169 [==============================] - 1291s 8s/step - loss: 2.0734 - bd_loss: 0.5353 - sn_loss: 0.5509 - mt_loss: 0.0671 - lt_loss: 0.0459 - oh_loss: 0.1707 - ch_loss: 0.5208 - ri_loss: 0.1191 - bd_accuracy: 0.7733 - sn_accuracy: 0.7362 - mt_accuracy: 0.9876 - l

KeyboardInterrupt: 

In [22]:
model.save('./chooseok/multi_output_l2_drp0.2.h5')

In [41]:
units = 64
dropout = 0.4

model = tf.keras.models.Sequential()
forward_layer = LSTM(units=units, dropout=dropout, recurrent_dropout=dropout)
backward_layer = LSTM(units=units, activation='relu', go_backwards=True, dropout=dropout, recurrent_dropout=dropout)
model.add(Bidirectional(forward_layer, backward_layer=backward_layer, input_shape=(63,22)))
model.add(Dense(22, activation='softmax'))

model.compile(optimizer=tf.keras.optimizers.Adam(), loss=tf.keras.losses.CategoricalCrossentropy(), metrics=['accuracy'])

In [4]:
train_x = []
train_y = []

for i in range(1,6000):
    tmp = pd.read_csv('./single_label_dataset/' + str(i) + '.csv.gzip', compression='gzip').to_numpy()

    train_x.append(tmp[:63])
    train_y.append(tmp[-1])

train_x = np.array(train_x)
train_y = np.array(train_y)



In [42]:
with tf.device('/gpu:0'):
    model.fit(x=train_x, y=train_y, epochs=5000, callbacks=callback, validation_split=0.1)

Epoch 1/5000
169/169 [==============================] - 115s 666ms/step - loss: 2.3419 - accuracy: 0.3221 - val_loss: 2.1286 - val_accuracy: 0.3417
Epoch 2/5000
169/169 [==============================] - 114s 672ms/step - loss: 2.0089 - accuracy: 0.3708 - val_loss: 1.9482 - val_accuracy: 0.3967
Epoch 3/5000
169/169 [==============================] - 115s 682ms/step - loss: 1.8927 - accuracy: 0.3867 - val_loss: 1.9487 - val_accuracy: 0.3850
Epoch 4/5000
169/169 [==============================] - 113s 668ms/step - loss: 1.8471 - accuracy: 0.3906 - val_loss: 1.8686 - val_accuracy: 0.4167
Epoch 5/5000
169/169 [==============================] - 117s 690ms/step - loss: 1.8441 - accuracy: 0.3980 - val_loss: 1.8245 - val_accuracy: 0.4267
Epoch 6/5000
169/169 [==============================] - 113s 668ms/step - loss: 1.7675 - accuracy: 0.4179 - val_loss: 1.8283 - val_accuracy: 0.4300
Epoch 7/5000
169/169 [==============================] - 112s 665ms/step - loss: 1.7366 - accuracy: 0.4264 - val_

KeyboardInterrupt: 

In [43]:
model.save('./chooseok/bidunits64dropout0.4.h5')

In [48]:
units = 128
dropout = 0.2

model = tf.keras.models.Sequential()
forward_layer = tf.keras.layers.GRU(units=units, dropout=dropout, recurrent_dropout=dropout, kernel_regularizer=l2(0.01), recurrent_regularizer=l2(0.01))
backward_layer = tf.keras.layers.GRU(units=units, activation='relu', go_backwards=True, dropout=dropout, recurrent_dropout=dropout, kernel_regularizer=l2(0.01), recurrent_regularizer=l2(0.01))
model.add(Bidirectional(forward_layer, backward_layer=backward_layer, input_shape=(63,7)))
model.add(Dense(7, activation='sigmoid'))

model.compile(optimizer=tf.keras.optimizers.Adam(), loss=tf.keras.losses.MSE, metrics=['accuracy'])

In [49]:
train_x.shape

(5999, 63, 7)

In [50]:
with tf.device('/gpu:0'):
    model.fit(x=train_x, y=train_y, epochs=5000, callbacks=callback, validation_split=0.1)

Epoch 1/5000
169/169 [==============================] - 109s 631ms/step - loss: 0.6602 - accuracy: 0.2189 - val_loss: 0.1167 - val_accuracy: 0.2133
Epoch 2/5000
169/169 [==============================] - 106s 629ms/step - loss: 0.1057 - accuracy: 0.2408 - val_loss: 0.1028 - val_accuracy: 0.3017
Epoch 3/5000
169/169 [==============================] - 107s 632ms/step - loss: 0.1025 - accuracy: 0.2625 - val_loss: 0.1021 - val_accuracy: 0.3067
Epoch 4/5000
169/169 [==============================] - 105s 624ms/step - loss: 0.1016 - accuracy: 0.2702 - val_loss: 0.1012 - val_accuracy: 0.2817
Epoch 5/5000
169/169 [==============================] - 106s 628ms/step - loss: 0.1008 - accuracy: 0.2854 - val_loss: 0.1012 - val_accuracy: 0.2600
Epoch 6/5000
169/169 [==============================] - 105s 624ms/step - loss: 0.1001 - accuracy: 0.2939 - val_loss: 0.1000 - val_accuracy: 0.3183
Epoch 7/5000
169/169 [==============================] - 107s 634ms/step - loss: 0.0993 - accuracy: 0.3012 - val_

In [51]:
model.save('./chooseok/GRU_MSE.h5')

In [5]:
train_x = []
train_y = []

for i in range(1,6000):
    tmp = pd.read_csv('./single_label_dataset/' + str(i) + '.csv.gzip', compression='gzip').to_numpy()
    train_x.append(tmp[:63])
    train_y.append(tmp[-1])

train_x = np.array(train_x)
train_y = np.array(train_y)



In [10]:
units = 128
dropout = 0.2

model = tf.keras.models.Sequential()
forward_layer = tf.keras.layers.GRU(units=units, dropout=dropout, recurrent_dropout=dropout)
backward_layer = tf.keras.layers.GRU(units=units, activation='relu', go_backwards=True, dropout=dropout, recurrent_dropout=dropout)
model.add(Bidirectional(forward_layer, backward_layer=backward_layer, input_shape=(63,22)))
model.add(Dense(22, activation='softmax'))

model.compile(optimizer=tf.keras.optimizers.Adam(), loss=tf.keras.losses.CategoricalCrossentropy(), metrics=['accuracy'])

In [11]:
with tf.device('/gpu:0'):
    model.fit(x=train_x, y=train_y, epochs=5000, callbacks=callback, validation_split=0.1)

Epoch 1/5000
169/169 [==============================] - 108s 614ms/step - loss: 2.0786 - accuracy: 0.3684 - val_loss: 1.8944 - val_accuracy: 0.4050
Epoch 2/5000
169/169 [==============================] - 105s 619ms/step - loss: 1.6942 - accuracy: 0.4527 - val_loss: 1.7084 - val_accuracy: 0.4433
Epoch 3/5000
169/169 [==============================] - 103s 609ms/step - loss: 1.6019 - accuracy: 0.4721 - val_loss: 1.6397 - val_accuracy: 0.4767
Epoch 4/5000
169/169 [==============================] - 104s 614ms/step - loss: 1.5528 - accuracy: 0.4827 - val_loss: 1.6152 - val_accuracy: 0.4817
Epoch 5/5000
169/169 [==============================] - 103s 612ms/step - loss: 1.5027 - accuracy: 0.5042 - val_loss: 1.5881 - val_accuracy: 0.5050
Epoch 6/5000
169/169 [==============================] - 103s 612ms/step - loss: 1.4794 - accuracy: 0.5145 - val_loss: 1.5423 - val_accuracy: 0.5100
Epoch 7/5000
169/169 [==============================] - 104s 613ms/step - loss: 1.4407 - accuracy: 0.5264 - val_

In [12]:
model.save('./chooseok/GRU_Cat_Single.h5')

------


In [ ]:
model.save('./chooseok/bidunits128dropout0.2l2norm.h5')

In [ ]:
units = 64
dropout = 0.4

encoder_inputs = Input(shape=(63,22))
encoder_lstm = LSTM(units=units, return_state=True, dropout=dropout, recurrent_dropout=dropout)

encoder_outputs, state_h, state_c = encoder_lstm(encoder_inputs)

encoder_states = [state_h, state_c]


decoder_inputs = Input(shape=(None,22))
decoder_lstm = LSTM(units=units, return_state=True, dropout=dropout, recurrent_dropout=dropout)

decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state = encoder_states)

decoder_dense = Dense(22, activation='softmax')

decoder_outputs = decoder_dense(decoder_outputs)


In [ ]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

callback = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=10)

model.compile(optimizer=tf.keras.optimizers.Adam(), loss=tf.keras.losses.CategoricalCrossentropy(), metrics=['accuracy'])

In [ ]:
with tf.device('/gpu:0'):
    model.fit(x=[train_x, dec], y=train_y, epochs=5000, callbacks=callback, validation_split=0.1)

In [ ]:
model.save('./chooseok/unit64dropout0.4.h5')

In [ ]:
units = 512
dropout = 0.4

encoder_inputs = Input(shape=(63,22))
encoder_lstm = LSTM(units=units, return_state=True, dropout=dropout, recurrent_dropout=dropout)

encoder_outputs, state_h, state_c = encoder_lstm(encoder_inputs)

encoder_states = [state_h, state_c]


decoder_inputs = Input(shape=(None,22))
decoder_lstm = LSTM(units=units, return_state=True, dropout=dropout, recurrent_dropout=dropout)

decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state = encoder_states)

decoder_dense = Dense(22, activation='softmax')

decoder_outputs = decoder_dense(decoder_outputs)


In [ ]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

callback = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=10)

model.compile(optimizer=tf.keras.optimizers.Adam(), loss=tf.keras.losses.CategoricalCrossentropy(), metrics=['accuracy'])

In [ ]:
with tf.device('/gpu:0'):
    model.fit(x=[train_x, dec], y=train_y, epochs=5000, callbacks=callback, validation_split=0.1)

In [ ]:
model.save('./chooseok/unit512dropout0.4.h5')